In [29]:
import pandas as pd
from scipy import stats
import numpy as np
from sklearn.feature_selection import mutual_info_classif

In [8]:
df: pd.DataFrame = pd.read_excel("../data/kredyty_auto_Scoring2025s.xlsx")

df.head()

,LP,data_akceptacji,grupa_ryzyka,kod_partnera,typ_umowy,scoring_FICO,okres_kredytu,kwota_kredytu,oproc_refin,oproc_konkur,koszt_pieniadza,oproc_propon,akceptacja_klienta
0,1,2005-04-17,2,2,R,702,48,26500.00,0.0399,0.0499,0.011,0.0659,0
1,2,2005-04-17,2,2,N,719,72,34999.99,0.0000,0.0479,0.011,0.0599,0
2,3,2005-04-17,1,3,N,802,60,28000.00,0.0000,0.0399,0.011,0.0355,0
3,4,2005-04-17,1,1,R,723,60,20750.69,0.0675,0.0499,0.011,0.0489,1
4,5,2005-04-17,3,1,U,690,72,19999.99,0.0000,0.0529,0.011,0.0679,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   LP                  40000 non-null  int64 
 1   data_akceptacji     40000 non-null  object
 2   grupa_ryzyka        40000 non-null  int64 
 3   kod_partnera        40000 non-null  int64 
 4   typ_umowy           40000 non-null  object
 5   scoring_FICO        40000 non-null  int64 
 6   okres_kredytu       40000 non-null  int64 
 7   kwota_kredytu       40000 non-null  object
 8   oproc_refin         40000 non-null  object
 9   oproc_konkur        40000 non-null  object
 10  koszt_pieniadza     40000 non-null  object
 11  oproc_propon        40000 non-null  object
 12  akceptacja_klienta  40000 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 4.0+ MB


In [10]:
df.describe()

,LP,data_akceptacji,grupa_ryzyka,kod_partnera,scoring_FICO,okres_kredytu,kwota_kredytu,oproc_refin,oproc_konkur,koszt_pieniadza,oproc_propon,akceptacja_klienta
count,40000.00000,40000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,20000.50000,2005-07-11 18:14:11.040000128,2.053725,2.103425,720.729175,57.186900,24211.651536,0.023644,0.049631,0.014169,0.065867,0.259175
min,1.00000,2005-03-25 00:00:00,1.000000,1.000000,601.000000,36.000000,4526.620000,0.000000,0.029900,0.010900,0.025900,0.000000
25%,10000.75000,2005-05-30 00:00:00,1.000000,1.000000,684.000000,48.000000,15703.592500,0.000000,0.044900,0.011138,0.048200,0.000000
50%,20000.50000,2005-07-12 00:00:00,2.000000,2.000000,717.000000,60.000000,23000.000000,0.000000,0.049900,0.013800,0.063400,0.000000
75%,30000.25000,2005-08-25 00:00:00,3.000000,3.000000,755.000000,60.000000,30906.825000,0.055000,0.053900,0.016300,0.082400,1.000000
max,40000.00000,2005-11-12 00:00:00,4.000000,3.000000,851.000000,72.000000,100000.000000,0.240000,0.062900,0.021010,0.113500,1.000000
std,11547.14972,NaN,1.090307,0.912825,46.580678,11.386207,11211.386637,0.041133,0.005794,0.002618,0.020704,0.438187


In [32]:
df["delta_oproc_konkurencji"] = df["oproc_propon"] - df["oproc_konkur"]
df["delta_oproc_refinansowana"] = df["oproc_propon"] - df["oproc_refin"]


In [ ]:
def calculate_annuit_rate(row):
    percentage = row["oproc_propon"]
    n = row["okres_kredytu"]

    if (
        pd.isnull(row["kwota_kredytu"])
        or pd.isnull(percentage)
        or pd.isnull(n)
        or n == 0
    ):
        return None  # obsługa braków danych lub dzielenia przez zero

    r = percentage / 12  # miesięczna stopa procentowa jako ułamek

    rata = row["kwota_kredytu"] * (r * (1 + r) ** n) / ((1 + r) ** n - 1)
    return rata


df["rata_annuita"] = df.apply(calculate_annuit_rate, axis=1)

In [ ]:
df["is_n"] = df.apply(lambda row: 1 if row["typ_umowy"] == "N" else 0, axis=1)
df["is_r"] = df.apply(lambda row: 1 if row["typ_umowy"] == "R" else 0, axis=1)

In [ ]:
def compute_woe_iv(
    df: pd.DataFrame, feature: str, target: str, bins: int = 10
) -> float:
    """Compute Information Value for a single feature."""
    series = df[feature]

    if pd.api.types.is_numeric_dtype(series):
        binned = pd.qcut(series, bins, duplicates="drop")
    else:
        binned = series.astype("object")

    tmp = pd.DataFrame({feature: binned, target: df[target]})
    grp = tmp.groupby(feature, observed=False)[target].agg(["count", "sum"])
    grp["non_event"] = grp["count"] - grp["sum"]

    grp["dist_event"] = grp["sum"] / grp["sum"].sum()
    grp["dist_non_event"] = grp["non_event"] / grp["non_event"].sum()

    grp = grp[(grp["dist_event"] > 0) & (grp["dist_non_event"] > 0)]
    grp["woe"] = np.log(grp["dist_event"] / grp["dist_non_event"])
    iv = ((grp["dist_event"] - grp["dist_non_event"]) * grp["woe"]).sum()
    return float(iv)


def cramers_v(x: pd.Series, y: pd.Series) -> float:
    """Compute Cramer's V statistic for association between two categorical series."""
    contingency = pd.crosstab(x, y)
    chi2, _, _, _ = stats.chi2_contingency(contingency)
    n = contingency.values.sum()
    phi2 = chi2 / n
    r, k = contingency.shape
    return float(np.sqrt(phi2 / min(k - 1, r - 1)))

In [ ]:
TARGET = "akceptacja_klienta"

EXCLUDED_FEATURES = ["LP"]

# run over all predictors
features = [c for c in df.columns if c not in EXCLUDED_FEATURES + [TARGET]]
iv_list = []
cv_list = []
for f in features:
    try:
        iv_list.append({"feature": f, "IV": compute_woe_iv(df, f, TARGET)})
        cv_list.append({"feature": f, "CramersV": cramers_v(df[f], df[TARGET])})
    except Exception as e:
        print(f"skipping {f}: {e}")

iv_df = pd.DataFrame(iv_list).sort_values("IV", ascending=False).reset_index(drop=True)
cv_df = (
    pd.DataFrame(cv_list)
    .sort_values("CramersV", ascending=False)
    .reset_index(drop=True)
)

In [24]:
iv_df

,feature,IV
0,kwota_kredytu,0.942273
1,typ_umowy,0.403734
2,oproc_refin,0.283355
3,oproc_konkur,0.213204
4,oproc_propon,0.210357
5,grupa_ryzyka,0.107380
6,scoring_FICO,0.084846
7,okres_kredytu,0.064450
8,data_akceptacji,0.038539
9,koszt_pieniadza,0.010345


In [25]:
cv_df

,feature,CramersV
0,kwota_kredytu,0.858116
1,oproc_propon,0.313878
2,oproc_refin,0.298565
3,typ_umowy,0.265231
4,oproc_konkur,0.243613
5,grupa_ryzyka,0.182717
6,kod_partnera,0.178668
7,scoring_FICO,0.163149
8,okres_kredytu,0.118698
9,data_akceptacji,0.085829


In [ ]:
X = df[features].apply(lambda col: pd.factorize(col, sort=True)[0])
y = df[TARGET].astype(int)

mi = mutual_info_classif(X, y, discrete_features="auto", random_state=42)
mi_df = pd.DataFrame({"feature": features, "MI": mi}).sort_values("MI", ascending=False)

mi_df

,feature,MI
6,kwota_kredytu,0.157796
10,oproc_propon,0.044854
3,typ_umowy,0.039235
7,oproc_refin,0.031582
8,oproc_konkur,0.030018
2,kod_partnera,0.018686
1,grupa_ryzyka,0.018082
5,okres_kredytu,0.011271
4,scoring_FICO,0.010907
0,data_akceptacji,0.001312
